# **Reduced order modeling for high contrast diffusion**

Our base model will be the diffusion equation:
$$-div(a(x) \nabla u(x)) = 1,\quad x \in [0, 1]^2,$$
with homogeneus Dirichlet boundary conditions. We take a piecewise constant diffusion coefficient
$$a_{|D_j} = y_j, \quad 1\leq j \leq 16,$$
on the subdomains
$$D_1 = [0, 0.25]^2, D_2 = [0.25, 0.5]\times[0, 0.25], \dots, D_{16} = [0.75, 1]^2.$$
Thus the solution $u$ depends on a parameter $y=(y_1,\dots,y_{16})$ which we pick in the parameter space 
$$Y= [1, \infty]^{16}.$$

Our goal is to study the effect on the approximation capabilities of reduced order methods when there is presence of high contrast diffusion coefficients $y_j=\infty$. In particular we study:
 * **Forward modeling** (or Galerkin projection): approximate a state $u(x,y)$ on the whole domain $x\in [0,1]^2$ for a given parameter $y$.
 * **State estimation**: approximate a state $u(x,y)$ on the whole domain $x\in [0,1]^2$ for some unknown parameter $y$, based on a few point measurements $u(x^i,y)$ at chosen points $x^1,\dots,x^m\in [0,1]^2$.
 * **$\mathbf{H_0^1}$ projection**: project a given state $u(x,y)$ the reduced basis subspace.
 * **Parameter estimation**: approximate the unknown parameter $y$, based on a few point measurements $u(x^i,y)$ at chosen points $x^1,\dots,x^m\in [0,1]^2$.

An offline stage is first needed in order to build the reduced basis that will be used afterwards to solve the three aforementioned problems:

1. **Compute $N$ solutions** (snapshots) $u^1=u(\cdot,y^1), \dots, u^N=u(\cdot, y^N)$ from a chosen set of parameters $\{y^1, \dots ,y^N\}\in Y^N$.

2. Create a **reduced basis** $V_n$ of dimension $n\leq N$ using the $N$ precomputed snapshots. This new space is generated by the solutions $V_n \subset span\{u^1,\dots,u^N \}$: 


## Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

src_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
project_path = os.path.abspath(os.path.join(src_path, os.pardir))
sys.path.insert(1, src_path)
sys.path.insert(1, project_path)

In [3]:
import matplotlib.pylab as plt
import numpy as np

from src.lib.VizUtils import plot_solutions_together
from lib.ReducedBasis import INFINIT_A, ReducedBasisGreedy, ReducedBasisRandom, ReducedBasisPCA, GREEDY_FOR_GALERKIN, GREEDY_FOR_H10
from src.experiments.HighContrast import experiment, plot_rates_of_convergence, type_of_problem_dict, get_full_a

import ipywidgets as widgets

np.random.seed(42)

## Parameters and definitions

In [4]:
diffusion_contrast_lower = 1 # a_min
diffusion_contrast_upper = INFINIT_A # a_max

#### Display parameters

In [5]:
num_points_per_dim_to_plot = 50 # size of the plotted images

## **_Offline stage_**

In [6]:
name = "jupyter_tests"

# --------- geometry ---------- #
blocks_geometry = (4, 4)
high_contrast_blocks=[
   [(0, 0), (1, 1), (3, 3)],
   [(0, 1), (1, 0), (2, 2)]
]

# --------- offline ---------- #
number_of_solutions = 200 # N
number_of_reduced_base_elements = 25  # n < N

reduced_basis_builders = [
    ReducedBasisRandom(add_inf_solutions=True),
    ReducedBasisRandom(add_inf_solutions=False),
    ReducedBasisGreedy(greedy_for=GREEDY_FOR_H10),
    ReducedBasisGreedy(greedy_for=GREEDY_FOR_GALERKIN),
    ReducedBasisPCA(add_inf_solutions=True),
    ReducedBasisPCA(add_inf_solutions=False),
]

reduced_basis_2show = [rb.name for rb in reduced_basis_builders]
inverse_dict = {v: k for k, v in type_of_problem_dict.items()}

# --------- online ---------- #
num_measurements = 100 # m >= n

In [7]:
sm, data = experiment(
    name=name, 
    reduced_basis_builders=reduced_basis_builders,
    mesh_discretization_per_dim=4,
    diff_coef_refinement= 30, 
    vn_max_dim=number_of_reduced_base_elements, 
    num_measurements=num_measurements,
    blocks_geometry=blocks_geometry,
    high_contrast_blocks=high_contrast_blocks, 
    vn_max_dim2do_stats=None,
    num_cores=1,
    max_num_samples_offline=number_of_solutions, 
    seed=42,
    recalculate_basis=True,
    recalculate=True,
    method="lsqsparse",
    verbose=False
)

print("The space V has dimension {}".format(sm.vspace_dim))

/home/callum/Repos/echantillonnageoptimal/venv/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:144: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',
Pre-calculating statistics.:   0%|          | 0/25 [00:00<?, ?it/s]/home/callum/Repos/echantillonnageoptimal/venv/lib/python3.8/site-packages/scipy/sparse/linalg/dsolve/linsolve.py:144: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn('spsolve requires A be CSC or CSR matrix format',
Pre-calculating statistics.: 100%|██████████| 25/25 [00:36<00:00,  1.46s/it]

The space V has dimension 225


In [8]:
def plot_convergence(reduced_basis, type_of_problems):
    fig, ax = plt.subplots(figsize=(10,8))
    plot_rates_of_convergence(
        ax, 
        data,
        reduced_basis_2show=reduced_basis, 
        type_of_problems=inverse_dict[type_of_problems], 
        color=None, 
        linestyle="solid",
        marker='.'
    )
    plt.show()

widgets.interact(
    plot_convergence,
    reduced_basis=widgets.SelectMultiple(value=reduced_basis_2show, options=reduced_basis_2show, description="Reduced basis: ", disabled=False),
    type_of_problems=widgets.Dropdown(value=r'$H_0^1$ projection', options=[r'$H_0^1$ projection', 'galerkin projection', 'state_estimation'], description="Problem: ", disabled=False),
)

            

interactive(children=(SelectMultiple(description='Reduced basis: ', index=(0, 1, 2, 3, 4, 5), options=('Random…

<function __main__.plot_convergence(reduced_basis, type_of_problems)>

In [9]:
def plot_estimation(n, reduced_basis, type_of_problem, **a_high_contrast):
    diffusion_coefficient = get_full_a(np.array([[a_high_contrast[k] for k in sorted(a_high_contrast.keys())]]), sm, high_contrast_blocks)
    true_solution = sm.generate_solutions(diffusion_coefficient)
    measurement_points = np.random.uniform(size=(num_measurements, 2))
    measurements = sm.evaluate_solutions(measurement_points, true_solution)
    
    solutions = [true_solution]
    for rbname in reduced_basis:
        rb = data[rbname]["basis"][:n]
        if inverse_dict[type_of_problem] == "forward_modeling":
            approx = rb.forward_modeling(sm, diffusion_coefficient)
        elif inverse_dict[type_of_problem] == "state_estimation": 
            approx = rb.state_estimation(sm, measurement_points, measurements, return_coefs=False)
        elif inverse_dict[type_of_problem] == "projection": 
            approx = rb.projection(sm, true_solution)
        else:
            raise Exception("Not implemented.")
        solutions.append(approx)
        
    plot_solutions_together(sm, None, solutions, num_points_per_dim_to_plot=100, contour_levels=7,
                            axes_xy_proportions=(2, 2), titles=["True solution"]+list(reduced_basis), colorbar=False)
    plt.show()

widgets.interact(
    plot_estimation,
    n=widgets.IntSlider(value=number_of_reduced_base_elements, max=number_of_reduced_base_elements, min=1),
    reduced_basis=widgets.SelectMultiple(value=reduced_basis_2show, options=reduced_basis_2show, description="Reduced basis: ", disabled=False),
    type_of_problem=widgets.Dropdown(value='$H_0^1$ projection', 
                                     options=['galerkin projection','$H_0^1$ projection', 'state_estimation'], description="Problem: ", disabled=False),
    **{f"a{i}": widgets.FloatLogSlider(value=1,min=np.log10(diffusion_contrast_lower),max=np.log10(diffusion_contrast_upper),step=0.1, description=f'a{i}:', disabled=False,
                           continuous_update=False, orientation='horizontal', readout=True, readout_format='.1f')
        for i in range(len(high_contrast_blocks))}
)

interactive(children=(IntSlider(value=25, description='n', max=25), SelectMultiple(description='Reduced basis:…

<function __main__.plot_estimation(n, reduced_basis, type_of_problem, **a_high_contrast)>